In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
!pip install opencv-python tensorflow==2.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 KB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: gast
    Found existing installa

In [3]:
cd drive/MyDrive/yolo_car_number_plate_detection/

/content/drive/MyDrive/yolo_car_number_plate_detection


In [4]:
import numpy as no 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2

In [5]:
cd labeling

/content/drive/MyDrive/yolo_car_number_plate_detection/labeling


In [16]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,images/N1.xml,1093,1396,645,727
1,images/N100.xml,134,301,312,350
2,images/N101.xml,31,139,128,161
3,images/N102.xml,164,316,216,243
4,images/N103.xml,813,1067,665,724


In [17]:
import xml.etree.ElementTree as xet

In [18]:
filename = df['filepath'][0]
filename

'images/N1.xml'

In [19]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [20]:
getFilename(filename)

'./images/N1.jpeg'

In [22]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images/N1.jpeg',
 './images/N100.jpeg',
 './images/N101.jpeg',
 './images/N102.jpeg',
 './images/N103.jpeg',
 './images/N104.jpeg',
 './images/N105.jpeg',
 './images/N106.jpeg',
 './images/N107.jpeg',
 './images/N108.jpeg',
 './images/N109.jpeg',
 './images/N11.jpeg',
 './images/N110.jpeg',
 './images/N111.jpeg',
 './images/N112.jpeg',
 './images/N113.jpeg',
 './images/N114.jpeg',
 './images/N116.jpeg',
 './images/N117.jpeg',
 './images/N118.jpeg',
 './images/N119.jpeg',
 './images/N12.jpeg',
 './images/N120.jpeg',
 './images/N121.jpeg',
 './images/N122.jpeg',
 './images/N123.jpeg',
 './images/N124.jpeg',
 './images/N126.jpeg',
 './images/N127.jpeg',
 './images/N128.jpeg',
 './images/N129.jpeg',
 './images/N130.jpeg',
 './images/N131.jpeg',
 './images/N132.jpeg',
 './images/N133.jpeg',
 './images/N134.jpeg',
 './images/N135.jpeg',
 './images/N136.jpeg',
 './images/N137.jpeg',
 './images/N138.jpeg',
 './images/N139.jpeg',
 './images/N14.jpeg',
 './images/N140.jpeg',
 './images/N141.

In [24]:
file_path = image_path[0]
file_path

'./images/N1.jpeg'

In [31]:
from google.colab.patches import cv2_imshow

img = cv2.imread(file_path)
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [35]:
cv2.rectangle(img, (1093, 645), (1396, 727), 3)
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

Data Preprocessing

In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [40]:
labels = df.iloc[:, 1:].values

In [47]:
data = []
output = []
for ind in range(len(image_path)):
  image = image_path[ind]
  img_arr = cv2.imread(image)
  h,w,d = img_arr.shape

  #preprocessing

  load_image = load_img(image, target_size=(224, 224))
  load_image_arr = img_to_array(load_image)
  norm_load_image_arr = load_image_arr/255.0 #normalization input

  #normalization to labels
  xmin, xmax, ymin, ymax = labels[ind]
  nxmin, nxmax = xmin/w, xmax/w
  nymin, nymax = ymin/h, ymax/w
  label_norm = (nxmin, nxmax, nymin, nymax) #normalization output

  data.append(norm_load_image_arr)
  output.append(label_norm)

In [48]:
import numpy as np
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [51]:
X.shape

(225, 224, 224, 3)

In [52]:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=0)

Deep Learning Model


In [54]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

In [55]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top = False, input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable = False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation="sigmoid")(headmodel)

model = Model(inception_resnet.input, outputs=headmodel)

219062272/219055592 [==============================] - 6s 0us/step


In [56]:
#complie model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [57]:
from tensorflow.keras.callbacks import TensorBoard
tfb = TensorBoard('object_detection')

In [60]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100, validation_data=(x_test,y_test), callbacks=[tfb], initial_epoch=8)

Epoch 9/100
18/18 [==============================] - 42s 2s/step - loss: 0.0617 - val_loss: 0.0657
Epoch 10/100
18/18 [==============================] - 40s 2s/step - loss: 0.0612 - val_loss: 0.0641
Epoch 11/100
18/18 [==============================] - 43s 2s/step - loss: 0.0613 - val_loss: 0.0630
Epoch 12/100
18/18 [==============================] - 40s 2s/step - loss: 0.0599 - val_loss: 0.0630
Epoch 13/100
18/18 [==============================] - 41s 2s/step - loss: 0.0594 - val_loss: 0.0640
Epoch 14/100
18/18 [==============================] - 39s 2s/step - loss: 0.0592 - val_loss: 0.0633
Epoch 15/100
18/18 [==============================] - 39s 2s/step - loss: 0.0588 - val_loss: 0.0636
Epoch 16/100
18/18 [==============================] - 42s 2s/step - loss: 0.0589 - val_loss: 0.0636
Epoch 17/100
18/18 [==============================] - 40s 2s/step - loss: 0.0594 - val_loss: 0.0644
Epoch 18/100
18/18 [==============================] - 39s 2s/step - loss: 0.0598 - val_loss: 0.0633
E

In [64]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200, validation_data=(x_test,y_test), callbacks=[tfb], initial_epoch=101)

Epoch 102/200
18/18 [==============================] - 47s 3s/step - loss: 4.9063e-04 - val_loss: 0.0072
Epoch 103/200
18/18 [==============================] - 46s 3s/step - loss: 5.1715e-04 - val_loss: 0.0080
Epoch 104/200
18/18 [==============================] - 44s 2s/step - loss: 7.2295e-04 - val_loss: 0.0074
Epoch 105/200
18/18 [==============================] - 43s 2s/step - loss: 9.6772e-04 - val_loss: 0.0077
Epoch 106/200
18/18 [==============================] - 42s 2s/step - loss: 8.6683e-04 - val_loss: 0.0073
Epoch 107/200
18/18 [==============================] - 46s 3s/step - loss: 6.9962e-04 - val_loss: 0.0074
Epoch 108/200
18/18 [==============================] - 46s 3s/step - loss: 7.3837e-04 - val_loss: 0.0079
Epoch 109/200
18/18 [==============================] - 42s 2s/step - loss: 5.8964e-04 - val_loss: 0.0076
Epoch 110/200
18/18 [==============================] - 47s 3s/step - loss: 4.6523e-04 - val_loss: 0.0071
Epoch 111/200
18/18 [==============================] - 

KeyboardInterrupt: ignored

In [65]:
model.save('./models/object_detection.h5')

In [71]:
!tensorboard --logdir="object_detection"

2023-03-03 09:35:20.237196: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-03-03 09:35:20.237234: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.11.2 at http://localhost:6006/ (Press CTRL+C to quit)
